### Parses PDF files and extracts numbered items from them into a list and then creates a github project with the list of items as issues.

This notebook reuses the code from the following notebooks:
- [ColPali_+_Qwen2_VL.ipynb](https://github.com/merveenoyan/smol-vision/blob/main/ColPali_%2B_Qwen2_VL.ipynb)

In [1]:
%pip install requests
%pip install python-dotenv
%pip install --upgrade byaldi
%pip install qwen-vl-utils
%pip install torchvision
# %pip install ipywidgets
# sudo apt-get install -y poppler-utils # To install dependency on linux

%pip install -q pdf2image git+https://github.com/huggingface/transformers.git flash-attn


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "/my_automations/.venv/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/my_automations/.venv/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/my_automations/.venv/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_proc

In [2]:

import os
import json
import torch
import requests
from dotenv import load_dotenv
from byaldi import RAGMultiModalModel
from pdf2image import convert_from_path
from huggingface_hub import notebook_login
from qwen_vl_utils import process_vision_info
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

# notebook_login()

# Load environment variables from .env file
load_dotenv()

False

In [10]:
import os

asset_dir = "my_automations/assets/"

files = [f for f in os.listdir(asset_dir) if os.path.isfile(os.path.join(asset_dir, f))]

print(files)

images = convert_from_path(asset_dir + files[0]) # TODO: loop through all files
print("Images length", len(images))

['doc1.pdf']


Images length 20


In [11]:

RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


adapter_config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [12]:
RAG.index(
    input_path= f"{asset_dir}{files[0]}", # path to the input file
    index_name="image_index", # index will be saved at index_root/index_name/
    store_collection_with_index=False,
    overwrite=True
)

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after tha

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


Added page 3 of document 0 to index.
Added page 4 of document 0 to index.


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


...

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


Added page 17 of document 0 to index.
Added page 18 of document 0 to index.


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Index exported to .byaldi/image_index
Index exported to .byaldi/image_index


{0: 'my_automations/assets/doc1.pdf'}

In [13]:
text_query = "What is the count of the items?"
results = RAG.search(text_query, k=1)
results

[{'doc_id': 0, 'page_num': 5, 'score': 16.25, 'metadata': {}, 'base64': None}]

In [14]:


model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct",
                                                        trust_remote_code=True, torch_dtype=torch.bfloat16).cuda().eval()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [15]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", trust_remote_code=True)

image_index = results[0]["page_num"] - 1
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": images[image_index],
            },
            {"type": "text", "text": text_query},
        ],
    }
]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [16]:

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)


In [17]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [18]:
generated_ids = model.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 22.15 GiB. GPU 0 has a total capacity of 23.64 GiB of which 11.02 GiB is free. Process 722470 has 12.62 GiB memory in use. Of the allocated memory 11.62 GiB is allocated by PyTorch, and 551.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(output_text)

In [ ]:
# Replace these variables with your own values
github_token =  os.getenv('GITHUB_PROJECT_SCOPE_PAT')
github_owner = os.getenv('GITHUB_OWNER')


token = github_token
owner = github_owner
project_name = 'First Project'
issue_title = 'Issue Title'
issue_body = 'Issue Body'

# Get the node ID of the owner
owner_url = f'https://api.github.com/users/{owner}'
headers = {
    'Authorization': f'token {token}',
    'Accept': 'application/vnd.github+json'
}
response = requests.get(owner_url, headers=headers)
owner_id = response.json()['node_id']
print("Owner ID:", owner_id)

In [ ]:

# Create the project
create_project_query = {
    'query': f'mutation {{createProjectV2(input: {{ownerId: "{owner_id}", title: "{project_name}"}}) {{projectV2 {{id}}}}}}'
}
response = requests.post('https://api.github.com/graphql', headers=headers, data=json.dumps(create_project_query))
print(response.json())
project_id = response.json()['data']['createProjectV2']['projectV2']['id']
print("Project ID:", project_id)

In [14]:
assignees = ['tunji17']  # GitHub usernames of the assignees

# Get the node IDs of the assignees
assignee_ids = []
for assignee in assignees:
    assignee_url = f'https://api.github.com/users/{assignee}'
    response = requests.get(assignee_url, headers=headers)
    assignee_ids.append(response.json()['node_id'])

assignees_str = ', '.join([f'"{assignee_id}"' for assignee_id in assignee_ids])

In [ ]:

# Add a draft issue to the project
add_issue_query = {
    'query': f'mutation {{addProjectV2DraftIssue(input: {{projectId: "{project_id}", title: "{issue_title}", body: "{issue_body}", assigneeIds: [{assignees_str}]}}) {{projectItem {{id}}}}}}'
}
response = requests.post('https://api.github.com/graphql', headers=headers, data=json.dumps(add_issue_query))
print(response.json())